# Types (estructuras *tipo*)

**NOTA:** Este notebook, y seguramente los que seguirán, serán escritos usando Julia v0.4-dev o v0.4-pre; algunas de los resultados que se obtienen abajo dependen de la versión que se utilice.

En Julia, **todo** es un objeto de algún *tipo*. Por ejemplo:

In [ ]:
typeof( 7 )

In [ ]:
typeof( 18612381238101715161911126301710 )

In [ ]:
typeof( 3//1)

In [ ]:
typeof( π )

In [ ]:
typeof( [1,2,3] )

In [ ]:
typeof((2,3.2))

In [ ]:
typeof( ["perro",2,3] )

In [ ]:
typeof( [[0, 1]  [1, 0]])

In [ ]:
typeof("π")

In [ ]:
f(x) = x^2-3
typeof(f)

In [ ]:
typeof(3.2 + 4im)

En Julia, realmente **todo** es un objeto de algún *tipo*.

In [ ]:
typeof(+)

La última intrucción *implica* que `+` es alguna función, y como tal, debe funcionar con ciertos tipos de entradas; a esto se le distingue como *métodos*. Para saber qué métodos están definidos en una función, usamos la *función* `methods`:

In [ ]:
methods(+)

El resultado de la última celda muestra que hay varios métodos (149) definidos en este momento en la sesión que estoy usando. Además, muestra un montón de otros tipos que no se habían mencionado.

El punto es que hay tipos concretos, como `Float64` o `Int32`, y *también* tipos abstractos, como `Number` o `Integer`.

Los tipos se organizan en un árbol jerárquico. Todo es de tipo `Any`, y la ausencia de 
tipo es `Void` (en Julia v0.3, `Void` es `Nothing`).

In [ ]:
isa(3.2,Any)

In [ ]:
isa(nothing,Void)

In [ ]:
nn() = return

typeof( nn() )

Para saber la estructura del árbol jerárquico, uno utiliza `super` para saber lo que está arriba, o `subtypes`

In [ ]:
super(Float64)

(En Julia v0.3, el resultado anterior es `FloatingPoint`.)

In [ ]:
super(AbstractFloat)

In [ ]:
super(Real)

In [ ]:
super(Number)

Y ahora los subtipos:

In [ ]:
subtypes(Integer)

En Julia v0.3 el resultado de la celda anterior es
```julia
5-element Array{Any,1}:
 BigInt  
 Bool    
 Char    
 Signed  
 Unsigned
```

In [ ]:
subtypes(Bool)

In [ ]:
subtypes(Number)

# Construyendo tipos propios (*tipos compuestos*)

En Julia, al igual que en todo lenguaje moderno que se precie de serlo, uno puede definir sus propias estructuras, lo que también se llama *tipos compuestos*. Esto da un manejo más versatil de los datos que definen las cantidades en que estamos interesados.

Para definir una nueva estructura de tipo usamos `type`.

Para ejemplificar esto, construiremos primero un vector de dos dimensiones, que llamaremos `Vec2Dtemp`

In [ ]:
type Vec2Dtemp
    x
    y
end

In [ ]:
Vec2Dtemp(1,2)

In [ ]:
ans.x, ans.y

La sutileza, que de hecho es un *problema*, de la definición anterior es que es *demasiado general*. Por ejemplo:

In [ ]:
Vec2Dtemp("perro", 1)

In [ ]:
ans.x

Quizás esto es lo que queremos, pero en general queremos darle cierta estructura definida a nuestros datos, cosa que en Julia significa ayudarle al compilador a hacer código eficiente. En este caso particular, nos interesa lidiar con números únicamente.

In [ ]:
type Vec2D{T<:Real}
    x :: T
    y :: T
end

In [ ]:
Vec2D(1.2,2.3)

In [ ]:
Vec2D(-6, 5)

Vale la pena notar que `Vec2D` distingue si sus elementos son `Int64` o `Float64`, u otra cosa..

Pero hay diferencias importantes:

In [ ]:
Vec2D("perro", 2)

In [ ]:
Vec2D(1.2, 2im)

Para resolver el último caso, usamos `promote`

In [ ]:
promote(1.2,2)

In [ ]:
Vec2D( promote(1.2,2) )

Aquí nos ayuda el operador *splat*, `...`, que son precisamente los (tres) puntos suspensivos:

In [ ]:
Vec2D( promote(1.2,2)... )

Para evitar hacer esto a cada rato, definimos un nuevo método para `Vec2D`

In [ ]:
Vec2D(a,b) = Vec2D(promote(a,b)...)

In [ ]:
Vec2D(1, 2.3)

Ahora, queremos *definir* operaciones que involucren varibles tipo `Vec2D`, en particular la suma:

In [ ]:
Vec2D(1,2) + Vec2D(0,3)

In [ ]:
# Lo primero sirve para *no* sobreescribir el operador `+` arbitrariamente
import Base.+
# ... y aquí se implementa la nueva función
+(a::Vec2D, b::Vec2D) = Vec2D( a.x + b.x, a.y + b.y )

**NOTA** No es necesario que la parte derecha incluya el mismo operador; puede corresponder a cualquier locura que se les ocurra...

In [ ]:
Vec2D(1,2) + Vec2D(0,3)

In [ ]:
Vec2D(1,2.2) + Vec2D(-0.1,0.3)

De igual manera se pueden definir los otros operadores, o incluso inventar nuevos. Un ejemplo es el siguiente:

In [ ]:
..(a::Real, b::Real) = Vec2D(a, b)

In [ ]:
..(1.2,3)

Y, lo anterior, en ciertos casos se puede escribir de manera más bonita:

In [ ]:
1.2 .. 3

In [ ]:
methods(..)

En particular, uno puede construir funciones genéricas que, si uno *sobrecarga* los operadores de manera adecuada, puede extender por ejemplo a `Vec2D`.

In [ ]:
f(x, y) = x + y